In [ ]:
import random
import numpy as np
from scipy import sparse
import random
# from sklearn.model_selection import train_test_split
from importlib import reload
from recapi import recommender as rec

In [ ]:
# Load the books dataset
import csv
incidence_file = '../data/csv/BX-Book-Ratings.csv'
user_item_rating_triples = list()
with open(incidence_file, encoding='latin1') as f:
    r = csv.reader(f, delimiter=';', skipinitialspace=True, quoting=csv.QUOTE_ALL)
    next(r)
    for row in r:
        if row[2] != '0' and row[2] < '5':
            continue
        user_item_rating_triples.append((row[0], row[1], 1))

In [ ]:
# movielens dataset
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')
user_item_rating_triples = [(u, i, 1) for u, i, r, t in data.raw_ratings if int(t) > 1]

In [ ]:
reload(rec)
%time rmdr = rec.build_recommender_from_triples(\
    user_item_rating_triples, min_item_freq=5, min_user_freq=3)

print('%d items left' % len(rmdr.items))
print('%d users left' % len(rmdr.users))

In [ ]:
reload(rec)
seed = random.randint(0,1000)
eval_str = 'mean relative rank %.4f, std %.3f'

seed = 15
alpha = 3
beta = 0.01

samples = 1000

%time ranks = rec.ranking_stat(rmdr, samples, seed, alpha=alpha, no_mat=False, no_bias=True)
print(eval_str % (np.mean(ranks), np.std(ranks)))

%time ranks = rec.ranking_stat(rmdr, samples, seed, alpha=alpha, beta=beta, no_mat=False, no_bias=False)
print(eval_str % (np.mean(ranks), np.std(ranks)))

%time ranks = rec.ranking_stat(rmdr, samples, seed, no_mat=True, no_bias=False)
print(eval_str % (np.mean(ranks), np.std(ranks)))

In [ ]:
reload(assessment)
from assessment import build_recommender, ranking_stat

eval_stat = dict()
eval_ranks = dict()

def register(params, typ, ranks):
    print(params)
    print(eval_str % (np.mean(ranks), np.std(ranks)))
    key = tuple(params + [typ])
    eval_stat[key] = (np.mean(ranks), np.std(ranks))
    eval_ranks[key] = ranks
    
seed = random.randint(0,1000)
eval_str = 'mean relative rank %.4f, std %.3f'

seed = 10
samples = 500

for tfidf in [True]:
    norm_param = 'tfidf' if tfidf else 'l2'
    rmdr = build_recommender(user_item_rating_triples, min_item_freq=5, min_user_freq=2)
    
    %time ranks = ranking_stat(rmdr, samples, seed, no_mat=True, no_bias=False)
    register([norm_param, None, None], 'nomat', ranks)

    for alpha in 3 * 1.2 ** np.arange(-2, 3):
        %time ranks = ranking_stat(rmdr, samples, seed, alpha=alpha, no_mat=False, no_bias=True)
        register([norm_param, '%3.2f' % alpha, None], 'nobias', ranks)

        for beta in 0.1 * 1.5 ** np.arange(-3, 4):
            params = ['tfidf' if tfidf else 'l2', '%3.1f' % alpha, '%3.1f' % beta]
            
            %time ranks = ranking_stat(rmdr, samples, seed, alpha=alpha, beta=beta, no_mat=False, no_bias=False)
            register([norm_param, '%3.2f' % alpha, '%3.2f' % beta], 'normal', ranks)

In [ ]:
import pickle
with open('./eval-seed3-muf2-mif5-samples1k.pickle', 'wb') as f:
    pickle.dump((eval_stat, eval_ranks), f)
    

In [ ]:
for r in sorted(eval_stat.items(), key=lambda x: x[1][0]):
    print(r[0], '%6.4f, std %4.2f' % r[1])